# dice (Dice)

In [ ]:
// // test

open testing

()



## sixth_power_sequence

In [ ]:
inl sixth_power_sequence () =
    stream.iterate ((*) 6) 1

()



In [ ]:
// // test

sixth_power_sequence ()
|> stream.take_while (fun _ i => i <= 7i32)
|> stream.to_list
|> _assert_eq [ 1i32; 6; 36; 216; 1296; 7776; 46656; 279936 ]

type UH0 =
    | UH0_0
    | UH0_1 of int32 * UH0
let rec method0 () : unit =
    let v0 : UH0 = UH0_0
    let v1 : UH0 = UH0_1(279936, v0)
    let v2 : UH0 = UH0_1(46656, v1)
    let v3 : UH0 = UH0_1(7776, v2)
    let v4 : UH0 = UH0_1(1296, v3)
    let v5 : UH0 = UH0_1(216, v4)
    let v6 : UH0 = UH0_1(36, v5)
    let v7 : UH0 = UH0_1(6, v6)
    let v8 : UH0 = UH0_1(1, v7)
    let v9 : string = $"%A{v8}"
    System.Console.WriteLine v9
    let v73 : UH0 = UH0_0
    let v74 : UH0 = UH0_1(279936, v73)
    let v75 : UH0 = UH0_1(46656, v74)
    let v76 : UH0 = UH0_1(7776, v75)
    let v77 : UH0 = UH0_1(1296, v76)
    let v78 : UH0 = UH0_1(216, v77)
    let v79 : UH0 = UH0_1(36, v78)
    let v80 : UH0 = UH0_1(6, v79)
    let v81 : UH0 = UH0_1(1, v80)
    let v82 : UH0 = UH0_0
    let v83 : UH0 = UH0_1(279936, v82)
    let v84 : UH0 = UH0_1(46656, v83)
    let v85 : UH0 = UH0_1(7776, v84)
    let v86 : UH0 = UH0_1(1296, v85)
    let v87 : UH0 = UH0_1(216, v86)
    let v88 : UH0 = UH0_1(36, 

## accumulate_dice_rolls

In [ ]:
inl accumulate_dice_rolls
    (log : option (string -> ()))
    (rolls : list u8)
    (power : i8)
    acc
    =
    inl rec body acc power rolls =
        match rolls with
        | _ when power < 0 =>
            inl result = acc + 1
            log |> optionm'.iter ((|>) $"$\"accumulate_dice_rolls / power: {!power} / acc: {!acc} / result: {!result}\"")
            Some (result, rolls)
        | [] => None
        | roll :: rest when roll > 1 =>
            inl coeff = sixth_power_sequence () |> stream.item power
            inl value = conv (roll - 1) * coeff
            log |> optionm'.iter ((|>) $"$\"accumulate_dice_rolls / power: {!power} / acc: {!acc} / roll: {!roll} / value: {!value}\""
            )
            loop (acc + value) (power - 1) rest
        | roll :: rest =>
            log |> optionm'.iter ((|>) $"$\"accumulate_dice_rolls / power: {!power} / acc: {!acc} / roll: {!roll}\"")
            loop acc (power - 1) rest
    and inl loop acc power rolls =
        if var_is rolls |> not
        then body acc power rolls
        else
            inl acc = dyn acc
            join body acc power rolls
    loop acc power rolls

()



In [ ]:
// // test

accumulate_dice_rolls (Some console.write_line) [ 6; 5; 4; 3; 2 ] 0 1000i32
|> _assert_eq (Some (1006, [ 5; 4; 3; 2 ]))

type UH0 =
    | UH0_0
    | UH0_1 of uint8 * UH0
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32 * f0_1 : UH0
    | US0_1
let rec method0 () : unit =
    let v0 : string = $"accumulate_dice_rolls / power: {0y} / acc: {1000} / roll: {6uy} / value: {5}"
    System.Console.WriteLine v0
    let v1 : string = $"accumulate_dice_rolls / power: {-1y} / acc: {1005} / result: {1006}"
    System.Console.WriteLine v1
    let v2 : UH0 = UH0_0
    let v3 : UH0 = UH0_1(2uy, v2)
    let v4 : UH0 = UH0_1(3uy, v3)
    let v5 : UH0 = UH0_1(4uy, v4)
    let v6 : UH0 = UH0_1(5uy, v5)
    let v7 : US0 = US0_0(1006, v6)
    let v8 : string = $"%A{v7}"
    System.Console.WriteLine v8
    let v42 : UH0 = UH0_0
    let v43 : UH0 = UH0_1(2uy, v42)
    let v44 : UH0 = UH0_1(3uy, v43)
    let v45 : UH0 = UH0_1(4uy, v44)
    let v46 : UH0 = UH0_1(5uy, v45)
    let v47 : US0 = US0_0(1006, v46)
    let v48 : UH0 = UH0_0
    let v49 : UH0 = UH0_1(2uy, v48)
    let v50 : UH0 = UH0_1(3uy, v49)
    let v51 : UH0 = UH0_

In [ ]:
// // test

accumulate_dice_rolls (Some console.write_line) [ 6; 5; 4; 3; 2 ] 1 1000i32
|> _assert_eq (Some (1035, [ 4; 3; 2 ]))

type UH0 =
    | UH0_0
    | UH0_1 of uint8 * UH0
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32 * f0_1 : UH0
    | US0_1
let rec method0 () : unit =
    let v0 : string = $"accumulate_dice_rolls / power: {1y} / acc: {1000} / roll: {6uy} / value: {30}"
    System.Console.WriteLine v0
    let v1 : string = $"accumulate_dice_rolls / power: {0y} / acc: {1030} / roll: {5uy} / value: {4}"
    System.Console.WriteLine v1
    let v2 : string = $"accumulate_dice_rolls / power: {-1y} / acc: {1034} / result: {1035}"
    System.Console.WriteLine v2
    let v3 : UH0 = UH0_0
    let v4 : UH0 = UH0_1(2uy, v3)
    let v5 : UH0 = UH0_1(3uy, v4)
    let v6 : UH0 = UH0_1(4uy, v5)
    let v7 : US0 = US0_0(1035, v6)
    let v8 : string = $"%A{v7}"
    System.Console.WriteLine v8
    let v34 : UH0 = UH0_0
    let v35 : UH0 = UH0_1(2uy, v34)
    let v36 : UH0 = UH0_1(3uy, v35)
    let v37 : UH0 = UH0_1(4uy, v36)
    let v38 : US0 = US0_0(1035, v37)
    let v39 : UH0 = UH0_0
    let v40 : UH0 = UH0_1(2uy, 

In [ ]:
// // test

accumulate_dice_rolls (Some console.write_line) [ 6; 5; 4; 3; 2 ] 2 1000i32
|> _assert_eq (Some (1208, [ 3; 2 ]))

type UH0 =
    | UH0_0
    | UH0_1 of uint8 * UH0
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32 * f0_1 : UH0
    | US0_1
let rec method0 () : unit =
    let v0 : string = $"accumulate_dice_rolls / power: {2y} / acc: {1000} / roll: {6uy} / value: {180}"
    System.Console.WriteLine v0
    let v1 : string = $"accumulate_dice_rolls / power: {1y} / acc: {1180} / roll: {5uy} / value: {24}"
    System.Console.WriteLine v1
    let v2 : string = $"accumulate_dice_rolls / power: {0y} / acc: {1204} / roll: {4uy} / value: {3}"
    System.Console.WriteLine v2
    let v3 : string = $"accumulate_dice_rolls / power: {-1y} / acc: {1207} / result: {1208}"
    System.Console.WriteLine v3
    let v4 : UH0 = UH0_0
    let v5 : UH0 = UH0_1(2uy, v4)
    let v6 : UH0 = UH0_1(3uy, v5)
    let v7 : US0 = US0_0(1208, v6)
    let v8 : string = $"%A{v7}"
    System.Console.WriteLine v8
    let v27 : UH0 = UH0_0
    let v28 : UH0 = UH0_1(2uy, v27)
    let v29 : UH0 = UH0_1(3uy, v28)
    let v30 : US0 = US0_0(12

## roll_within_bounds

In [ ]:
inl roll_within_bounds (log : option (string -> ())) max rolls =
    inl power = listm.length rolls - 1
    match accumulate_dice_rolls log rolls power 0 with
    | Some (result, _) when result >= 1 && result <= max => Some result
    | _ => None

In [ ]:
// // test

roll_within_bounds (Some console.write_line) 2000i32 [ 1; 5; 4; 4; 5 ]
|> _assert_eq (Some 995)

type [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
let rec method0 () : unit =
    let v0 : string = $"accumulate_dice_rolls / power: {4y} / acc: {0} / roll: {1uy}"
    System.Console.WriteLine v0
    let v1 : string = $"accumulate_dice_rolls / power: {3y} / acc: {0} / roll: {5uy} / value: {864}"
    System.Console.WriteLine v1
    let v2 : string = $"accumulate_dice_rolls / power: {2y} / acc: {864} / roll: {4uy} / value: {108}"
    System.Console.WriteLine v2
    let v3 : string = $"accumulate_dice_rolls / power: {1y} / acc: {972} / roll: {4uy} / value: {18}"
    System.Console.WriteLine v3
    let v4 : string = $"accumulate_dice_rolls / power: {0y} / acc: {990} / roll: {5uy} / value: {4}"
    System.Console.WriteLine v4
    let v5 : string = $"accumulate_dice_rolls / power: {-1y} / acc: {994} / result: {995}"
    System.Console.WriteLine v5
    let v6 : US0 = US0_0(995)
    let v7 : string = $"%A{v6}"
    System.Console.WriteLine v7
    let v11 : US0 = US0_0(995)
    let v

In [ ]:
// // test

roll_within_bounds (Some console.write_line) 2000i32 [ 2; 2; 6; 4; 5 ]
|> _assert_eq (Some 1715)

type [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
let rec method0 () : unit =
    let v0 : string = $"accumulate_dice_rolls / power: {4y} / acc: {0} / roll: {2uy} / value: {1296}"
    System.Console.WriteLine v0
    let v1 : string = $"accumulate_dice_rolls / power: {3y} / acc: {1296} / roll: {2uy} / value: {216}"
    System.Console.WriteLine v1
    let v2 : string = $"accumulate_dice_rolls / power: {2y} / acc: {1512} / roll: {6uy} / value: {180}"
    System.Console.WriteLine v2
    let v3 : string = $"accumulate_dice_rolls / power: {1y} / acc: {1692} / roll: {4uy} / value: {18}"
    System.Console.WriteLine v3
    let v4 : string = $"accumulate_dice_rolls / power: {0y} / acc: {1710} / roll: {5uy} / value: {4}"
    System.Console.WriteLine v4
    let v5 : string = $"accumulate_dice_rolls / power: {-1y} / acc: {1714} / result: {1715}"
    System.Console.WriteLine v5
    let v6 : US0 = US0_0(1715)
    let v7 : string = $"%A{v6}"
    System.Console.WriteLine v7
    let v11 : U

In [ ]:
// // test

roll_within_bounds (Some console.write_line) 2000i32 [ 4; 1; 1; 2; 3 ]
|> _assert_eq None

type [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
let rec method0 () : unit =
    let v0 : string = $"accumulate_dice_rolls / power: {4y} / acc: {0} / roll: {4uy} / value: {3888}"
    System.Console.WriteLine v0
    let v1 : string = $"accumulate_dice_rolls / power: {3y} / acc: {3888} / roll: {1uy}"
    System.Console.WriteLine v1
    let v2 : string = $"accumulate_dice_rolls / power: {2y} / acc: {3888} / roll: {1uy}"
    System.Console.WriteLine v2
    let v3 : string = $"accumulate_dice_rolls / power: {1y} / acc: {3888} / roll: {2uy} / value: {6}"
    System.Console.WriteLine v3
    let v4 : string = $"accumulate_dice_rolls / power: {0y} / acc: {3894} / roll: {3uy} / value: {2}"
    System.Console.WriteLine v4
    let v5 : string = $"accumulate_dice_rolls / power: {-1y} / acc: {3896} / result: {3897}"
    System.Console.WriteLine v5
    let v6 : US0 = US0_1
    let v7 : string = $"%A{v6}"
    System.Console.WriteLine v7
    let v11 : US0 = US0_1
    let v12 : US0 = US0_1


## calculate_dice_count

In [ ]:
inl calculate_dice_count (log : option (string -> ())) max =
    inl rec body n p =
        inl return () =
            log |> optionm'.iter ((|>) $"$\"calculate_dice_count / max: {!max} / n: {!n} / p: {!p}\"")
            n

        if p < max then
            inl p' = p * 6
            if p' > p
            then loop (n + 1) p'
            else return ()
        else return ()

    and inl loop n p =
        if var_is max |> not
        then body n p
        else
            inl n = dyn n
            inl p = dyn p
            join body n p
    if max = 1
    then 1
    else loop 0 1

()



In [ ]:
// // test

calculate_dice_count (Some console.write_line) 36i32
|> _assert_eq 2i32

let rec method0 () : unit =
    let v0 : string = $"calculate_dice_count / max: {36} / n: {2} / p: {36}"
    System.Console.WriteLine v0
    let v1 : string = $"%A{2}"
    System.Console.WriteLine v1
    let v2 : string = $"__expect / actual: %A{2} / expected: %A{2}"
    ()
method0()

calculate_dice_count / max: 36 / n: 2 / p: 36
2


In [ ]:
// // test

calculate_dice_count (Some console.write_line) 7777i32
|> _assert_eq 6i32

let rec method0 () : unit =
    let v0 : string = $"calculate_dice_count / max: {7777} / n: {6} / p: {46656}"
    System.Console.WriteLine v0
    let v1 : string = $"%A{6}"
    System.Console.WriteLine v1
    let v2 : string = $"__expect / actual: %A{6} / expected: %A{6}"
    ()
method0()

calculate_dice_count / max: 7777 / n: 6 / p: 46656
6


## roll_dice

In [ ]:
inl roll_dice () : u8 =
    run_target function
        | Rust (Contract) => fun () => rust.emit_expr () "1"
        | Rust _ => fun () => rust.emit_expr () "rand::Rng::gen_range(&mut rand::thread_rng(), 1..7)"
        | _ => fun () => $"System.Random().Next (1, 7) |> uint8"

()



## rotate_number

In [ ]:
inl rotate_number forall t {number}. (max : i64) (n : t) : t =
    (conv n - 1 + max) % max + 1 |> conv

()



## rotate_numbers

In [ ]:
inl rotate_numbers max items =
    items |> stream.map (rotate_number max)

()



In [ ]:
// // test

listm'.init_series -1 14 1i32
|> stream.from_list
|> rotate_numbers 6
|> stream.to_list
|> _assert_eq [ 5; 6; 1; 2; 3; 4; 5; 6; 1; 2; 3; 4; 5; 6; 1; 2 ]

type UH0 =
    | UH0_0
    | UH0_1 of int32 * UH0
let rec method0 () : unit =
    let v0 : UH0 = UH0_0
    let v1 : UH0 = UH0_1(2, v0)
    let v2 : UH0 = UH0_1(1, v1)
    let v3 : UH0 = UH0_1(6, v2)
    let v4 : UH0 = UH0_1(5, v3)
    let v5 : UH0 = UH0_1(4, v4)
    let v6 : UH0 = UH0_1(3, v5)
    let v7 : UH0 = UH0_1(2, v6)
    let v8 : UH0 = UH0_1(1, v7)
    let v9 : UH0 = UH0_1(6, v8)
    let v10 : UH0 = UH0_1(5, v9)
    let v11 : UH0 = UH0_1(4, v10)
    let v12 : UH0 = UH0_1(3, v11)
    let v13 : UH0 = UH0_1(2, v12)
    let v14 : UH0 = UH0_1(1, v13)
    let v15 : UH0 = UH0_1(6, v14)
    let v16 : UH0 = UH0_1(5, v15)
    let v17 : string = $"%A{v16}"
    System.Console.WriteLine v17
    let v205 : UH0 = UH0_0
    let v206 : UH0 = UH0_1(2, v205)
    let v207 : UH0 = UH0_1(1, v206)
    let v208 : UH0 = UH0_1(6, v207)
    let v209 : UH0 = UH0_1(5, v208)
    let v210 : UH0 = UH0_1(4, v209)
    let v211 : UH0 = UH0_1(3, v210)
    let v212 : UH0 = UH0_1(2, v211)
    let v213 : UH0 = UH0_1

## create_sequential_roller

In [ ]:
inl create_sequential_roller (log : option (string -> ())) list =
    log |> optionm'.iter ((|>) $"$\"create_sequential_roller ()\"")
    inl s = list |> listm.rev |> listm.append list |> stream.from_list |> stream.memoize
    inl current_index = mut 0i64
    inl acc = mut 1
    inl len = mut -1
    inl last_item = mut None
    let rec loop () =
        fun () =>
            inl current_index = *current_index
            inl acc = *acc
            inl len = *len
            inl last_item = *last_item
            log |> optionm'.iter ((|>) $"$\"create_sequential_roller / roll / current_index: {!current_index} / acc: {!acc} / len: {!len} / last_item: %A{!last_item}\"")
        |> fun x => x ()
        match s () |> stream.try_item *current_index with
        | Some item =>
            current_index <- *current_index + 1
            last_item <- Some item
            item
        | None =>
            log |> optionm'.iter ((|>) $"$\"create_sequential_roller / roll / None\"")
            if *len = -1
            then len <- *current_index
            acc <-
                if *acc >= *len
                then 1
                else *acc + 1
            current_index <- *acc - 1
            last_item <- None
            loop ()
    loop

()



In [ ]:
// // test

inl sequential_roll = create_sequential_roller (Some console.write_line) [ 1i32; 2; 3; 4 ]

am.init 50i32 (ignore >> sequential_roll)
|> _assert_eq (a ;[ 1; 2; 3; 4; 4; 3; 2; 1; 2; 3; 4; 4; 3; 2; 1; 3; 4; 4; 3; 2; 1; 4; 4; 3; 2; 1;
4; 3; 2; 1; 3; 2; 1; 2; 1; 1; 1; 2; 3; 4; 4; 3; 2; 1; 2; 3; 4; 4; 3; 2 ] : a i32 i32)

type UH0 =
    | UH0_0 of int32 * (unit -> UH0)
    | UH0_1
and [<Struct>] US0 =
    | US0_0 of f0_0 : (unit -> UH0)
    | US0_1 of f1_0 : UH0
and Mut0 = {mutable l0 : US0}
and Mut1 = {mutable l0 : int64}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int32
    | US1_1
and Mut2 = {mutable l0 : US1}
and Mut3 = {mutable l0 : int32}
let rec closure8 () () : UH0 =
    UH0_1
and closure7 () () : UH0 =
    let v0 : (unit -> UH0) = closure8()
    UH0_0(1, v0)
and closure6 () () : UH0 =
    let v0 : (unit -> UH0) = closure7()
    UH0_0(2, v0)
and closure5 () () : UH0 =
    let v0 : (unit -> UH0) = closure6()
    UH0_0(3, v0)
and closure4 () () : UH0 =
    let v0 : (unit -> UH0) = closure5()
    UH0_0(4, v0)
and closure3 () () : UH0 =
    let v0 : (unit -> UH0) = closure4()
    UH0_0(4, v0)
and closure2 () () : UH0 =
    let v0 : (unit -> UH0) = closure3()
    UH0_0(3, v0)
and closure1 () () : UH0 =
    let v0 : (unit -> UH0) = closure2()
    UH0_0(2, v0)
and closure0 () () : UH0 =
    let v0 : (un

## roll_progressively

In [ ]:
inl roll_progressively (log : option (string -> ())) roll reroll max =
    inl power = (calculate_dice_count log max) - 1
    let rec loop rolls size =
        if size < power + 1
        then loop (roll () :: rolls) (size + 1)
        else accumulate_dice_rolls log rolls power 0 |> function
            | Some (result, _) when result <= max => result
            | _ when reroll => loop (listm.init power (fun _ => roll ())) power
            | _ => loop (roll () :: rolls) (size + 1)
    loop [] 0

()



In [ ]:
// // test

roll_progressively None roll_dice false 1
|> _assert_eq 1i32

type UH0 =
    | UH0_0
    | UH0_1 of uint8 * UH0
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32 * f0_1 : UH0
    | US0_1
let rec method3 (v0 : UH0, v1 : int32) : US0 =
    let v2 : int32 = v1 + 1
    let v3 : string = $"accumulate_dice_rolls / power: {-1y} / acc: {v1} / result: {v2}"
    US0_0(v2, v0)
and method2 (v0 : UH0, v1 : int32) : US0 =
    match v0 with
    | UH0_1(v3, v4) -> (* Cons *)
        let v5 : bool = v3 > 1uy
        if v5 then
            let v6 : uint8 = v3 - 1uy
            let v7 : int32 = int32 v6
            let v8 : string = $"accumulate_dice_rolls / power: {0y} / acc: {v1} / roll: {v3} / value: {v7}"
            let v9 : int32 = v1 + v7
            method3(v4, v9)
        else
            let v11 : string = $"accumulate_dice_rolls / power: {0y} / acc: {v1} / roll: {v3}"
            method3(v4, v1)
    | UH0_0 -> (* Nil *)
        US0_1
and method1 (v0 : UH0, v1 : int8) : int32 =
    let v2 : bool = v1 < 1y
    if v2 then
        let v3 : uint8 option = None

In [ ]:
// // test

inl sequential_roll = create_sequential_roller (Some console.write_line) [ 5; 4; 4; 5; 1 ]

roll_progressively (Some console.write_line) sequential_roll false 2000i32
|> _assert_eq 995

type UH0 =
    | UH0_0 of uint8 * (unit -> UH0)
    | UH0_1
and [<Struct>] US0 =
    | US0_0 of f0_0 : (unit -> UH0)
    | US0_1 of f1_0 : UH0
and Mut0 = {mutable l0 : US0}
and Mut1 = {mutable l0 : int64}
and [<Struct>] US1 =
    | US1_0 of f0_0 : uint8
    | US1_1
and Mut2 = {mutable l0 : US1}
and UH1 =
    | UH1_0
    | UH1_1 of uint8 * UH1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int32 * f0_1 : UH1
    | US2_1
let rec closure10 () () : UH0 =
    UH0_1
and closure9 () () : UH0 =
    let v0 : (unit -> UH0) = closure10()
    UH0_0(5uy, v0)
and closure8 () () : UH0 =
    let v0 : (unit -> UH0) = closure9()
    UH0_0(4uy, v0)
and closure7 () () : UH0 =
    let v0 : (unit -> UH0) = closure8()
    UH0_0(4uy, v0)
and closure6 () () : UH0 =
    let v0 : (unit -> UH0) = closure7()
    UH0_0(5uy, v0)
and closure5 () () : UH0 =
    let v0 : (unit -> UH0) = closure6()
    UH0_0(1uy, v0)
and closure4 () () : UH0 =
    let v0 : (unit -> UH0) = closure5()
    UH0_0(1uy, v0)
and closure3 () () : U

In [ ]:
// // test

inl sequential_roll = create_sequential_roller (Some console.write_line) [ 5; 4; 4; 5; 2 ]

roll_progressively (Some console.write_line) sequential_roll false 2000i32
|> _assert_eq 1678

type UH0 =
    | UH0_0 of uint8 * (unit -> UH0)
    | UH0_1
and [<Struct>] US0 =
    | US0_0 of f0_0 : (unit -> UH0)
    | US0_1 of f1_0 : UH0
and Mut0 = {mutable l0 : US0}
and Mut1 = {mutable l0 : int64}
and [<Struct>] US1 =
    | US1_0 of f0_0 : uint8
    | US1_1
and Mut2 = {mutable l0 : US1}
and UH1 =
    | UH1_0
    | UH1_1 of uint8 * UH1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int32 * f0_1 : UH1
    | US2_1
let rec closure10 () () : UH0 =
    UH0_1
and closure9 () () : UH0 =
    let v0 : (unit -> UH0) = closure10()
    UH0_0(5uy, v0)
and closure8 () () : UH0 =
    let v0 : (unit -> UH0) = closure9()
    UH0_0(4uy, v0)
and closure7 () () : UH0 =
    let v0 : (unit -> UH0) = closure8()
    UH0_0(4uy, v0)
and closure6 () () : UH0 =
    let v0 : (unit -> UH0) = closure7()
    UH0_0(5uy, v0)
and closure5 () () : UH0 =
    let v0 : (unit -> UH0) = closure6()
    UH0_0(2uy, v0)
and closure4 () () : UH0 =
    let v0 : (unit -> UH0) = closure5()
    UH0_0(2uy, v0)
and closure3 () () : U

In [ ]:
// // test

join dyn listm'.init_series 1i32 10000 1
|> listm.map (fun _ => roll_progressively None roll_dice false 10i32)
|> listm'.group_by id
|> listm.map (fun (k, v) => k, (listm.length v : i32))
|> listm.toArray
|> fun (ar : a i32 _) =>
    $"!ar |> Array.sortBy (fun (struct (a, b)) -> a)" : a i32 $"struct (int32 * int32)"

index value 0 (1, 287) Item1 1 Item2 287 1 (2, 706) Item1 2 Item2 706 2 (3, 1327) Item1 3 Item2 1327 3 (4, 1328) Item1 4 Item2 1328 4 (5, 1366) Item1 5 Item2 1366 5 (6, 1324) Item1 6 Item2 1324 6 (7, 267) Item1 7 Item2 267 7 (8, 706) Item1 8 Item2 706 8 (9, 1352) Item1 9 Item2 1352 9 (10, 1337) Item1 10 Item2 1337

type UH0 =
    | UH0_0
    | UH0_1 of int32 * UH0
and UH1 =
    | UH1_0
    | UH1_1 of uint8 * UH1
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32 * f0_1 : UH1
    | US0_1
and UH2 =
    | UH2_0
    | UH2_1 of int32 * UH0 * UH2
and UH3 =
    | UH3_0
    | UH3_1 of int32 * int32 * UH3
let rec method2 (v0 : float, v1 : int32, v2 : int32, v3 : float) : UH0 =
    let v4 : bool = v3 < v0
    if v4 then
        let v5 : int32 = int32 v3
        let v6 : int32 = v1 * v5
        let v7 : int32 = v2 + v6
        let v8 : float = v3 + 1.0
        let v9 : UH0 = method2(v0, v1, v2, v8)
        UH0_1(v7, v9)
    else
        UH0_0
and closure2 (v0 : int32, v1 : int32) (v2 : int32) : UH0 =
    let v3 : int32 = v1 - v0
    let v4 : int32 = v3 / v2
    let v5 : float = float v4
    let v6 : float = v5 + 1.0
    let v7 : float = 0.0
    method2(v6, v2, v0, v7)
and closure1 (v0 : int32) (v1 : int32) : (int32 -> UH0) =
    closure2(v0, v1)
and closure0 () (v0 : int32) : (int32 -> (int32 -> UH0)) =
    c

In [ ]:
// // test
// // print_code=false

join dyn listm'.init_series 1i32 10000 1
|> listm.map (fun _ => roll_progressively None roll_dice true 10i32)
|> listm'.group_by id
|> listm.map (fun (k, v) => k, (listm.length v : i32))
|> listm.toArray
|> fun (ar : a i32 _) =>
    $"!ar |> Array.sortBy (fun (struct (a, b)) -> a)" : a i32 $"struct (int32 * int32)"

index value 0 (1, 1012) Item1 1 Item2 1012 1 (2, 1000) Item1 2 Item2 1000 2 (3, 981) Item1 3 Item2 981 3 (4, 962) Item1 4 Item2 962 4 (5, 1021) Item1 5 Item2 1021 5 (6, 1002) Item1 6 Item2 1002 6 (7, 1007) Item1 7 Item2 1007 7 (8, 1018) Item1 8 Item2 1018 8 (9, 1015) Item1 9 Item2 1015 9 (10, 982) Item1 10 Item2 982

In [ ]:
// // test
// // timeout=60000

join dyn listm'.init_series 1i32 100 1
|> listm.iter fun n =>
    listm'.init_series 0i32 1 1
    |> listm.iter fun reroll =>
        join dyn listm'.init_series 1i32 3500 1
        |> listm.map fun _ => roll_progressively None roll_dice (reroll = 1) n
        |> listm'.group_by id
        |> listm.length
        |> _assert_eq n

type UH0 =
    | UH0_0
    | UH0_1 of int32 * UH0
and UH1 =
    | UH1_0
    | UH1_1 of uint8 * UH1
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32 * f0_1 : UH1
    | US0_1
and UH2 =
    | UH2_0 of int32 * (unit -> UH2)
    | UH2_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : int32
    | US1_1
and UH3 =
    | UH3_0
    | UH3_1 of int32 * UH0 * UH3
let rec method2 (v0 : float, v1 : int32, v2 : int32, v3 : float) : UH0 =
    let v4 : bool = v3 < v0
    if v4 then
        let v5 : int32 = int32 v3
        let v6 : int32 = v1 * v5
        let v7 : int32 = v2 + v6
        let v8 : float = v3 + 1.0
        let v9 : UH0 = method2(v0, v1, v2, v8)
        UH0_1(v7, v9)
    else
        UH0_0
and closure2 (v0 : int32, v1 : int32) (v2 : int32) : UH0 =
    let v3 : int32 = v1 - v0
    let v4 : int32 = v3 / v2
    let v5 : float = float v4
    let v6 : float = v5 + 1.0
    let v7 : float = 0.0
    method2(v6, v2, v0, v7)
and closure1 (v0 : int32) (v1 : int32) : (int32 -> UH0) =
    closure2(v0, v1)
a

## main

In [ ]:
inl main (_args : array_base string) =
    inl result = roll_progressively (Some console.write_line) roll_dice true (limit.max : i64)
    console.write_line ($"$\"result: {!result}\"" : string)
    0i32

inl main () =
    inl rotate_numbers' : i64 -> stream.stream u8 -> stream.stream u8 =
        rotate_numbers

    inl create_sequential_roller' :
        optionm'.option' (string -> ())
        -> list u8 
        -> (() -> u8)
        =
        optionm'.unbox >> create_sequential_roller

    inl roll_progressively' :
        optionm'.option' (string -> ()) 
        -> (() -> u8)
        -> bool
        -> u64
        -> u64
        =
        optionm'.unbox >> roll_progressively

    inl roll_within_bounds' :
        optionm'.option' (string -> ())
        -> u64
        -> list u8
        -> optionm'.option' u64
        =
        fun a b c =>
            (optionm'.unbox >> roll_within_bounds) a b c
            |> optionm'.box

    $"let rotate_numbers x = !rotate_numbers' x" : ()
    $"let create_sequential_roller x = !create_sequential_roller' x" : ()
    $"let roll_progressively x = !roll_progressively' x" : ()
    $"let roll_within_bounds x = !roll_within_bounds' x" : ()
    $"let main args = !main args" : ()

type UH0 =
    | UH0_0 of uint8 * (unit -> UH0)
    | UH0_1
and UH1 =
    | UH1_0
    | UH1_1 of uint8 * UH1
and [<Struct>] US0 =
    | US0_0 of f0_0 : (string -> unit)
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : (unit -> UH0)
    | US1_1 of f1_0 : UH0
and Mut0 = {mutable l0 : US1}
and Mut1 = {mutable l0 : int64}
and [<Struct>] US2 =
    | US2_0 of f0_0 : uint8
    | US2_1
and Mut2 = {mutable l0 : US2}
and [<Struct>] US3 =
    | US3_0 of f0_0 : uint64 * f0_1 : UH1
    | US3_1
and UH2 =
    | UH2_0 of uint64 * (unit -> UH2)
    | UH2_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : uint64
    | US4_1
and [<Struct>] US5 =
    | US5_0 of f0_0 : int64 * f0_1 : UH1
    | US5_1
let rec closure2 (v0 : UH0) () : UH0 =
    v0
and method0 (v0 : int64, v1 : UH0, v2 : UH0) : UH0 =
    match v1 with
    | UH0_0(v3, v4) -> (* StreamCons *)
        let v5 : UH0 = v4 ()
        let v6 : UH0 = method0(v0, v5, v2)
        let v7 : int64 = int64 v3
        let v8 : int64 = v7 - 1L
        let v9 :